In [93]:
#%matplotlib inline
%load_ext autoreload
%autoreload 1
import numpy as np
import numpy.linalg as la
import scipy.sparse as sp
from scipy.sparse.linalg import norm
import util as util
import petsc_io as io
import matplotlib
import matplotlib.pyplot as plt
import model
import pod
import math
from decimal import Decimal
np.set_printoptions(threshold=np.nan)
np.set_printoptions(precision=20)
%aimport pod
%aimport petsc_io
%aimport model
%aimport util

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


metos3dbgc(dt,q,t,y,u,bc,dc,n=shape(q,1),nz=shape(q,0),m=len(u),nbc=len(bc),ndc=shape(dc,1))
integer :: n, nz, m, nbc, ndc
real*8  :: dt, q(nz, n), t, y(nz, n), u(m), bc(nbc), dc(nz, ndc)

In [3]:
Y_N = pod.constructSnapshotMatrix("simulation/POD/N-Model",1,200,7,6)
U_POD_N , s_POD_N ,V_POD_N = pod.computeSVD(Y_N)

Data loaded


In [4]:
Y_N = pod.constructSnapshotMatrix("simulation/DEIM/N-Model",1,200,7,6)
U_DEIM_N , s_DEIM_N ,V_DEIM_N = pod.computeSVD(Y_N)

Data loaded


In [6]:
J,PJ = pod.generateIndicesForNonlinearFunction()
#PT,p = pod.deimInterpolationIndices(U_DEIM.toarray())
PT_N,p_N = pod.deimInterpolationIndices(U_DEIM_N[:,:100])

In [134]:
np.save("reduced_basis/300_12/p.npy", p_N)

In [5]:
pod.plotSingular(s_POD_N,0.000001)
pod.plotSingular(s_DEIM_N,0.000001)

108

In [10]:
np.save('U_POD_N', U_POD_N)
np.save('s_POD_N', s_POD_N)
np.save('U_DEIM_N', U_DEIM_N)
np.save('s_DEIM_N', s_DEIM_N)
np.save('P_matrix', P_matrix)
np.save('PT', PT_N)
np.save('p', p_N)

Data loaded


In [ ]:
pod.plotSingular(s_deim,0.000001)
np.save('U_deim', U_deim)
np.save('U_deim', U_deim)
U_deim_m = U_deim[:,:80]
print(U_deim.shape, U_deim_m.shape)

In [39]:
lsm = io.read_PETSc_mat("data/TMM/2.8/Geometry/landSeaMask.petsc")
x = np.zeros(p_N.shape)
y = np.zeros(p_N.shape)
offset = 0
index = 0
for ix in range(64):
    for iy in range(128):
        length = lsm[ix, iy]
        if not length == 0:
            for ip in p_N:
                if offset <= ip and (offset + length) > ip:
#                   print(offset,ip,length)
                    x[index] = ix
                    y[index] = iy
                    index = index + 1
            offset = offset + length

In [42]:
plt.scatter(x,y)
plt.show()

In [8]:
PU_inv = la.inv(np.dot(PT_N.T,U_DEIM_N[:,:100]))
P_matrix = np.dot(U_DEIM_N[:,:100],PU_inv)
pod.createReducedMatrices(U_POD_N[:,:150],P_matrix)

In [ ]:
k= 450
m = 400
J,PJ = pod.generateIndicesForNonlinearFunction()
p = np.load("p.npy")
U_pod_k = np.load("U_pod.npy")[:,:k]
U_deim_m = np.load("U_deim.npy")[:,:m]

In [13]:
pod.plotErrorNorm(1,24,24,1)


Data loaded
Data loaded


In [12]:
pod.reducedModelDeimN(1000,2880,PJ,J,p_N,U_POD_N[:,:150],U_DEIM_N[:,:100])

projection error:  [ 2.15704254112873305971  2.1543446564900934348   2.1739998450266337926
  2.17594882189332405886  2.15503033229015183991  2.15967339731881136089
  2.1733378720404905593   2.17605808329927041811  2.15461845144474084535
  2.16511916490764733112  2.1732804503141953667   2.17599793861514445581
  2.15405710289324892059  2.16858904844919031518  2.17435907851820653391
  2.17615029552683125758  2.15329150300346050528  2.16923068509025895878
  2.17605956357948615576  2.1767617279481665804   2.15220996928095065215
  2.16787275723099348568  2.17745649732773216556  2.17778662232150432843
  2.15148336393453831761  2.16609018952622767173  2.17793964048522914467
  2.17879534613175573909  2.15129702683536061514  2.16494031791013652466
  2.17743204005893353781  2.17921862715944003597  2.15162588152778466721
  2.16499412641150490444  2.17634188977720999247  2.17896354635231981334
  2.15357304249842496091  2.16487806596772447421  2.17518812256178506459
  2.17819189562331150256  2.15520

KeyboardInterrupt: 

In [ ]:
i=0
Ai00 = io.read_PETSc_mat('data/TMM/2.8/Transport/Matrix5_4/1dt/Ai_'+str(i).zfill(2)+'.petsc')
#Ae00 = io.read_PETSc_mat('data/TMM/2.8/Transport/Matrix5_4/1dt/Ae_'+str(i).zfill(2)+'.petsc')
#io.write_PETSc_mat(Ae00,'Ae_test.petsc')
io.write_PETSc_mat(Ai00,'Ai_test.petsc')
#Ae00_t = io.read_PETSc_mat('Ae_test.petsc')
Ai00_t = io.read_PETSc_mat('Ai_test.petsc')

np.zeros_like
print(Ai00, norm(Ai00- Ai00_t))#,Ae00-Ae00_t)

In [ ]:
for i,l in PJ[p]:
    print("i: ", i ,j )

In [ ]:
spin = 0
s = 2
v1_n = io.read_PETSc_vec('simulation/compare/sp' + str(spin).zfill(4) + 'ts'+  str(s).zfill(4) + 'N.petsc')
v2_n = io.read_PETSc_vec('simulation/compare/' + '/sp'+str(spin).zfill(4)+'ts'+str(s).zfill(4)+'N_python.petsc')
v1_dop = io.read_PETSc_vec('simulation/compare/sp' + str(spin).zfill(4) + 'ts' + str(s).zfill(4)+ 'DOP.petsc')
v2_dop = io.read_PETSc_vec('simulation/compare/' + '/sp'+str(spin).zfill(4)+'ts'+str(s).zfill(4)+'DOP_python.petsc')
y = J[np.nonzero(np.sum(fice, axis=1))]
x = (v1_dop- v2_dop)[y]
print(np.linalg.norm(v1_n-v2_n), np.linalg.norm(v1_dop-v2_dop), np.nonzero(v1_dop- v2_dop),(v1_dop- v2_dop)[np.nonzero(v1_dop- v2_dop)],J[np.nonzero(np.sum(fice, axis=1))])

In [ ]:
 v1 = io.read_PETSc_vec('simulation/compare/sp' + str(spin).zfill(4) + 'ts'+  str(s).zfill(4) + 'N.petsc')
#io.write_PETSc_vec(y[:,0], 'simulation/compare/' + '/sp'+str(spin).zfill(4)+'ts'+str(s).zfill(4)+'N_python.petsc')
v2 = io.read_PETSc_vec('simulation/compare/sp' + str(spin).zfill(4) + 'ts' + str(s).zfill(4)+ 'DOP.petsc')
#io.write_PETSc_vec(y[:,1], 'simulation/compare/' + '/sp'+str(spin).zfill(4)+'ts'+str(s).zfill(4)+'DOP_python.petsc')
print("spin: ", spin,"step: ", s,"t:", t,"norm: ", str(s).zfill(4) ,np.linalg.norm(y_long[:offset]-v1), np.linalg.norm(y_long[offset:]-v2))

In [ ]:
gen.generate_prameter_samples(1)

In [ ]:
v = io.read_PETSc_vec('simulation/linear/sp0007ts2879N_01_UD.petsc')
v1 = io.read_PETSc_vec('simulation/linear/PO4_InitialData_01.petsc')
#print(np.mean(v),max(v),min(v1),max(v1),v,v1, np.linalg.norm(v),np.sum(v),np.sum(v1))
np.linalg.norm(v-v1)

In [48]:
# longitude
dx = 2.8125
xx = np.concatenate([np.arange(-180 + 0.5 * dx, 0, dx), np.arange(0 + 0.5 * dx, 180, dx)])
# latitude
dy = 2.8125
yy = np.arange(-90 + 0.5 * dy, 90, dy)
cmin, cmax = [0.0, 4.5]


# range of latitude
yl, yr = [6, 58]
# aspect of axes
aspect = (yy[yr] - yy[yl] + 1) / 5200.0 / 1.8
# levels
levels = np.arange(2.0, 3, 0.01)
#    print yl, yr
#    print aspect
#    print levels

v3d = io.read_data('POD/sp0000ts0004N.petsc')

slices = [73, 117, 32]
#title = "Pacific"
#yvisible = True
#create_figure_slice(1, aspect, yy, yl, yr, slices[0], title, yvisible, cmin, cmax, levels, v3d)
#title = "Atlantic"
#yvisible = True
#create_figure_slice(2, aspect, yy, yl, yr, slices[1], title, yvisible, cmin, cmax, levels, v3d)
#title = "Indian"
#yvisible = True
#create_figure_slice(3, aspect, yy, yl, yr, slices[2], title, yvisible, cmin, cmax, levels, v3d)
aspect = 1.0
io.create_figure_surface(4, aspect, xx, yy, cmin, cmax, levels, slices, v3d)

/home/njp/Master/Masterthesis/work/petsc_io.py:163: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  v3d[0, ix, iy, 0:length] = v1d[offset:offset+length]
/home/njp/Master/Masterthesis/work/petsc_io.py:164: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  v3d[1, ix, iy, 0:length] = z[offset:offset+length]
/home/njp/Master/Masterthesis/work/petsc_io.py:165: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  v3d[2, ix, iy, 0:length] = dz[offset:offset+length]


In [59]:
import configparser
config = configparser.ConfigParser()
config['Model'] = {'Metos3DProfileInputDirectory': 'data/TMM/2.8/Geometry/',
                   'Metos3DTracerOutputDirectory': 'work/simulation/',
                   'Metos3DParameterCount':'7' 
                   'Metos3DParameterValue':'0.02,2.0,0.5,30.0,0.67,0.5,0.858'
                } 

for i in range(10):
    x = "sp%.4dts%.4dN.petsc" % (i,i)
    print(x)

sp0000ts0000N.petsc
sp0001ts0001N.petsc
sp0002ts0002N.petsc
sp0003ts0003N.petsc
sp0004ts0004N.petsc
sp0005ts0005N.petsc
sp0006ts0006N.petsc
sp0007ts0007N.petsc
sp0008ts0008N.petsc
sp0009ts0009N.petsc


In [131]:
config = util.parse_config('option.txt')
print(np.array(config.get("-Metos3DParameterValue"),dtype=np.float_))
lsm = io.read_PETSc_mat("data/TMM/2.8/Geometry/landSeaMask.petsc")

['2']
['Profile']
['data/TMM/2.8/Geometry/']
['landSeaMask.petsc']
['volumes.petsc']
['2']
['2.17e+0', '1.e-4']
['work/simulation/POD/']
['N.petsc', 'DOP.petsc']
['7']
['0.02', '2.0', '0.5', '30.0', '0.67', '0.5', '0.858']
['2']
['data/TMM/2.8/Forcing/BoundaryCondition/']
['Latitude', 'IceCover']
['1']
['latitude.petsc']
['12']
['fice_%02d.petsc']
['2']
['data/TMM/2.8/Forcing/DomainCondition/']
['LayerDepth', 'LayerHeight']
['1']
['z.petsc']
['1']
['dz.petsc']
['Matrix']
['data/TMM/2.8/Transport/Matrix5_4/1dt/']
['12']
['Ae_%02d.petsc']
['Ai_%02d.petsc']
['0.0']
['2880']
['0.0003472222222222']
['Spinup']
['3000']
['sp%0004d', 'ts%0004d']
['1', '1']
[  2.00000000000000004163e-02   2.00000000000000000000e+00
   5.00000000000000000000e-01   3.00000000000000000000e+01
   6.70000000000000039968e-01   5.00000000000000000000e-01
   8.57999999999999984901e-01]


In [132]:
lsm.shap

(64, 128)